# <span style="color:brown;">Clean Column Names</span>

In [70]:
import polars as pl
from polars import col as c
import re

pl_mkt = pl.read_csv("data/marketing_data.csv")

print(pl_mkt.head())

print(pl_mkt.columns)

shape: (5, 26)
┌──────┬──────┬─────────────┬─────────────┬───┬─────────────┬─────────────┬───────────┬────────────┐
│ week ┆ Year ┆ Market.Shar ┆ Av.Price.pe ┆ … ┆ Reach.cinem ┆ GRP.outdoor ┆ GRP.print ┆ Share.of.s │
│ ---  ┆ ---  ┆ e           ┆ r.kg        ┆   ┆ a           ┆ ---         ┆ ---       ┆ pend       │
│ i64  ┆ i64  ┆ ---         ┆ ---         ┆   ┆ ---         ┆ i64         ┆ f64       ┆ ---        │
│      ┆      ┆ f64         ┆ f64         ┆   ┆ f64         ┆             ┆           ┆ f64        │
╞══════╪══════╪═════════════╪═════════════╪═══╪═════════════╪═════════════╪═══════════╪════════════╡
│ 19   ┆ 2010 ┆ 38.4        ┆ 7.61        ┆ … ┆ null        ┆ null        ┆ null      ┆ null       │
│ 20   ┆ 2010 ┆ 36.8        ┆ 7.6         ┆ … ┆ null        ┆ null        ┆ null      ┆ null       │
│ 21   ┆ 2010 ┆ 35.21       ┆ 7.63        ┆ … ┆ null        ┆ null        ┆ null      ┆ null       │
│ 22   ┆ 2010 ┆ 35.03       ┆ 7.22        ┆ … ┆ null        ┆ null        ┆ 

In [71]:
pl_mkt = (
    pl_mkt
    .rename(lambda col: re.sub('\\.|\\s+', '_', col.lower()))
)

print(pl_mkt)

print(pl_mkt.columns)

shape: (156, 26)
┌──────┬──────┬─────────────┬─────────────┬───┬─────────────┬─────────────┬───────────┬────────────┐
│ week ┆ year ┆ market_shar ┆ av_price_pe ┆ … ┆ reach_cinem ┆ grp_outdoor ┆ grp_print ┆ share_of_s │
│ ---  ┆ ---  ┆ e           ┆ r_kg        ┆   ┆ a           ┆ ---         ┆ ---       ┆ pend       │
│ i64  ┆ i64  ┆ ---         ┆ ---         ┆   ┆ ---         ┆ i64         ┆ f64       ┆ ---        │
│      ┆      ┆ f64         ┆ f64         ┆   ┆ f64         ┆             ┆           ┆ f64        │
╞══════╪══════╪═════════════╪═════════════╪═══╪═════════════╪═════════════╪═══════════╪════════════╡
│ 19   ┆ 2010 ┆ 38.4        ┆ 7.61        ┆ … ┆ null        ┆ null        ┆ null      ┆ null       │
│ 20   ┆ 2010 ┆ 36.8        ┆ 7.6         ┆ … ┆ null        ┆ null        ┆ null      ┆ null       │
│ 21   ┆ 2010 ┆ 35.21       ┆ 7.63        ┆ … ┆ null        ┆ null        ┆ null      ┆ null       │
│ 22   ┆ 2010 ┆ 35.03       ┆ 7.22        ┆ … ┆ null        ┆ null        

# <span style="color:brown;">Handle NA Values</span>

In [72]:
print(
    pl_mkt
    .null_count()
    .transpose(
        include_header=True,
        header_name="column",
        column_names=["null_count"]
    )
)

shape: (26, 2)
┌────────────────────────┬────────────┐
│ column                 ┆ null_count │
│ ---                    ┆ ---        │
│ str                    ┆ u32        │
╞════════════════════════╪════════════╡
│ week                   ┆ 0          │
│ year                   ┆ 0          │
│ market_share           ┆ 0          │
│ av_price_per_kg        ┆ 0          │
│ non-promo_price_per_kg ┆ 0          │
│ …                      ┆ …          │
│ reach_tv               ┆ 104        │
│ reach_cinema           ┆ 138        │
│ grp_outdoor            ┆ 155        │
│ grp_print              ┆ 134        │
│ share_of_spend         ┆ 40         │
└────────────────────────┴────────────┘


In [86]:
# Drop columns where the NA values are more than 1/3 of the total observations

cols_to_drop = (
    pl_mkt
    .null_count()
    .transpose(
        include_header=True,
        header_name="column",
        column_names=["null_count"]
    )
    .filter(c('null_count') > (1/3 * pl_mkt.shape[0]))
    .get_column('column')
)

print(cols_to_drop)

shape: (7,)
Series: 'column' [str]
[
	"grp_radio"
	"reach_radio"
	"grp_tv"
	"reach_tv"
	"reach_cinema"
	"grp_outdoor"
	"grp_print"
]


In [89]:
pl_mkt_dropped = pl_mkt.drop(*cols_to_drop)

print(pl_mkt_dropped)

shape: (156, 19)
┌──────┬──────┬──────────────┬───────────────┬───┬───────┬─────────────┬────────────┬──────────────┐
│ week ┆ year ┆ market_share ┆ av_price_per_ ┆ … ┆ aided ┆ penetration ┆ competitor ┆ share_of_spe │
│ ---  ┆ ---  ┆ ---          ┆ kg            ┆   ┆ ---   ┆ ---         ┆ ---        ┆ nd           │
│ i64  ┆ i64  ┆ f64          ┆ ---           ┆   ┆ f64   ┆ f64         ┆ f64        ┆ ---          │
│      ┆      ┆              ┆ f64           ┆   ┆       ┆             ┆            ┆ f64          │
╞══════╪══════╪══════════════╪═══════════════╪═══╪═══════╪═════════════╪════════════╪══════════════╡
│ 19   ┆ 2010 ┆ 38.4         ┆ 7.61          ┆ … ┆ null  ┆ null        ┆ null       ┆ null         │
│ 20   ┆ 2010 ┆ 36.8         ┆ 7.6           ┆ … ┆ null  ┆ null        ┆ null       ┆ null         │
│ 21   ┆ 2010 ┆ 35.21        ┆ 7.63          ┆ … ┆ null  ┆ null        ┆ null       ┆ null         │
│ 22   ┆ 2010 ┆ 35.03        ┆ 7.22          ┆ … ┆ null  ┆ null        ┆ n

In [90]:
# Fill NA values with "mean" strategy

print(pl_mkt_dropped.fill_null(strategy = 'mean'))

shape: (156, 19)
┌──────┬──────┬─────────────┬─────────────┬───┬───────────┬─────────────┬────────────┬─────────────┐
│ week ┆ year ┆ market_shar ┆ av_price_pe ┆ … ┆ aided     ┆ penetration ┆ competitor ┆ share_of_sp │
│ ---  ┆ ---  ┆ e           ┆ r_kg        ┆   ┆ ---       ┆ ---         ┆ ---        ┆ end         │
│ i64  ┆ i64  ┆ ---         ┆ ---         ┆   ┆ f64       ┆ f64         ┆ f64        ┆ ---         │
│      ┆      ┆ f64         ┆ f64         ┆   ┆           ┆             ┆            ┆ f64         │
╞══════╪══════╪═════════════╪═════════════╪═══╪═══════════╪═════════════╪════════════╪═════════════╡
│ 19   ┆ 2010 ┆ 38.4        ┆ 7.61        ┆ … ┆ 98.201626 ┆ 64.346341   ┆ 2.924324   ┆ 45.314027   │
│ 20   ┆ 2010 ┆ 36.8        ┆ 7.6         ┆ … ┆ 98.201626 ┆ 64.346341   ┆ 2.924324   ┆ 45.314027   │
│ 21   ┆ 2010 ┆ 35.21       ┆ 7.63        ┆ … ┆ 98.201626 ┆ 64.346341   ┆ 2.924324   ┆ 45.314027   │
│ 22   ┆ 2010 ┆ 35.03       ┆ 7.22        ┆ … ┆ 98.201626 ┆ 64.346341   ┆ 

In [94]:
# Fill NA values using machine learning KNNImputer from sklearn

from sklearn.impute import KNNImputer

pl_mkt_filled = pl.DataFrame(
    data = KNNImputer(n_neighbors = 10).fit_transform(pl_mkt_dropped.to_numpy()),
    schema = pl_mkt_dropped.columns
)

print(pl_mkt_filled)

shape: (156, 19)
┌──────┬────────┬──────────────┬──────────────┬───┬───────┬─────────────┬────────────┬─────────────┐
│ week ┆ year   ┆ market_share ┆ av_price_per ┆ … ┆ aided ┆ penetration ┆ competitor ┆ share_of_sp │
│ ---  ┆ ---    ┆ ---          ┆ _kg          ┆   ┆ ---   ┆ ---         ┆ ---        ┆ end         │
│ f64  ┆ f64    ┆ f64          ┆ ---          ┆   ┆ f64   ┆ f64         ┆ f64        ┆ ---         │
│      ┆        ┆              ┆ f64          ┆   ┆       ┆             ┆            ┆ f64         │
╞══════╪════════╪══════════════╪══════════════╪═══╪═══════╪═════════════╪════════════╪═════════════╡
│ 19.0 ┆ 2010.0 ┆ 38.4         ┆ 7.61         ┆ … ┆ 98.3  ┆ 0.0         ┆ 0.63       ┆ 60.680307   │
│ 20.0 ┆ 2010.0 ┆ 36.8         ┆ 7.6          ┆ … ┆ 98.3  ┆ 0.0         ┆ 0.63       ┆ 60.680307   │
│ 21.0 ┆ 2010.0 ┆ 35.21        ┆ 7.63         ┆ … ┆ 98.3  ┆ 0.0         ┆ 0.63       ┆ 60.680307   │
│ 22.0 ┆ 2010.0 ┆ 35.03        ┆ 7.22         ┆ … ┆ 98.3  ┆ 0.0         ┆ 

In [97]:
# Fill NA values using machine learning IterativeImputer from sklearn

from sklearn.experimental import enable_iterative_imputer  # This enables the experimental feature
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

pl_mkt_filled = pl.DataFrame(
    data = IterativeImputer(max_iter=10, random_state=0, estimator=RandomForestRegressor()).fit_transform(pl_mkt_dropped.to_numpy()),
    schema = pl_mkt_dropped.columns
)

print(pl_mkt_filled)

shape: (156, 19)
┌──────┬────────┬──────────────┬─────────────┬───┬────────┬─────────────┬────────────┬─────────────┐
│ week ┆ year   ┆ market_share ┆ av_price_pe ┆ … ┆ aided  ┆ penetration ┆ competitor ┆ share_of_sp │
│ ---  ┆ ---    ┆ ---          ┆ r_kg        ┆   ┆ ---    ┆ ---         ┆ ---        ┆ end         │
│ f64  ┆ f64    ┆ f64          ┆ ---         ┆   ┆ f64    ┆ f64         ┆ f64        ┆ ---         │
│      ┆        ┆              ┆ f64         ┆   ┆        ┆             ┆            ┆ f64         │
╞══════╪════════╪══════════════╪═════════════╪═══╪════════╪═════════════╪════════════╪═════════════╡
│ 19.0 ┆ 2010.0 ┆ 38.4         ┆ 7.61        ┆ … ┆ 98.055 ┆ 0.0         ┆ 0.701      ┆ 62.992447   │
│ 20.0 ┆ 2010.0 ┆ 36.8         ┆ 7.6         ┆ … ┆ 98.121 ┆ 0.0         ┆ 0.722      ┆ 70.518443   │
│ 21.0 ┆ 2010.0 ┆ 35.21        ┆ 7.63        ┆ … ┆ 98.099 ┆ 0.0         ┆ 0.727      ┆ 45.115148   │
│ 22.0 ┆ 2010.0 ┆ 35.03        ┆ 7.22        ┆ … ┆ 98.081 ┆ 0.0         ┆ 

# <span style="color:brown;">Binning - Discretization</span>